In [2]:
import pandas as pd

In [19]:
df_data = pd.read_csv('./data.csv', index_col='Unnamed: 0')
df_data.head(4)

,Desc,Unnamed: 0.1,author,genre,image_link,rating,title
0,We know that power is shifting: From West to E...,0.0,Moisés Naím,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.63,The End of Power: From Boardrooms to Battlefie...
1,Following the success of The Accidental Billio...,1.0,Blake J. Harris,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.94,"Console Wars: Sega, Nintendo, and the Battle t..."
2,How to tap the power of social software and ne...,2.0,Chris Brogan,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.78,Trust Agents: Using the Web to Build Influence...
3,William J. Bernstein is an American financial ...,3.0,William J. Bernstein,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.20,The Four Pillars of Investing


In [20]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2382 entries, 0 to 2450
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Desc          2382 non-null   object 
 1   Unnamed: 0.1  1185 non-null   float64
 2   author        2382 non-null   object 
 3   genre         2382 non-null   object 
 4   image_link    2382 non-null   object 
 5   rating        2382 non-null   float64
 6   title         2382 non-null   object 
dtypes: float64(2), object(5)
memory usage: 148.9+ KB


In [21]:
df_data['Desc']

0       We know that power is shifting: From West to E...
1       Following the success of The Accidental Billio...
2       How to tap the power of social software and ne...
3       William J. Bernstein is an American financial ...
4       Amazing book. And I joined Steve Jobs and many...
                              ...                        
2446    Ralph Roberts, a sus setenta años y tras la mu...
2447    Murder at the Vicarage marks the debut of Agat...
2448    In 1951 John Wyndham published his novel The D...
2449    This now classic book revealed Flannery O'Conn...
2450    Imbued on every page with Frank McCourt's asto...
Name: Desc, Length: 2382, dtype: object

In [22]:
from nltk.tokenize import RegexpTokenizer

In [23]:
text01 = 'We know that power is shifting: From West to E'
def remove_punctuation(text):
  tokenizer = RegexpTokenizer('[a-zA-Z]+')
  text_list = tokenizer.tokenize(text)
  # print(type(text_list), text_list)
  result = ' '.join(text_list)
  # print(type(result), result)
  return result

In [26]:
# for text01 in df_data['Desc']:
remove_punctuation(text01)

'We know that power is shifting From West to E'

In [27]:
df_data['cleaned'] = df_data['Desc'].apply(remove_punctuation)

In [28]:
df_data.columns

Index(['Desc', 'Unnamed: 0.1', 'author', 'genre', 'image_link', 'rating',
       'title', 'cleaned'],
      dtype='object')

In [29]:
df_data['cleaned'].head(5)

0    We know that power is shifting From West to Ea...
1    Following the success of The Accidental Billio...
2    How to tap the power of social software and ne...
3    William J Bernstein is an American financial t...
4    Amazing book And I joined Steve Jobs and many ...
Name: cleaned, dtype: object

In [31]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2382 entries, 0 to 2450
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Desc          2382 non-null   object 
 1   Unnamed: 0.1  1185 non-null   float64
 2   author        2382 non-null   object 
 3   genre         2382 non-null   object 
 4   image_link    2382 non-null   object 
 5   rating        2382 non-null   float64
 6   title         2382 non-null   object 
 7   cleaned       2382 non-null   object 
dtypes: float64(2), object(6)
memory usage: 167.5+ KB


##Pre-Trained

In [32]:
!curl -O https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1570M  100 1570M    0     0  36.0M      0  0:00:43  0:00:43 --:--:-- 32.5M


In [33]:
from gensim.models import Word2Vec

In [46]:
corpus = []
for words in df_data['cleaned']:
  corpus.append(words.split())
print(type(corpus), corpus[4:8])

<class 'list'> [['Amazing', 'book', 'And', 'I', 'joined', 'Steve', 'Jobs', 'and', 'many', 'more', 'from', 'the', 'Akio', 'Morita', 'fan', 'club', 'Did', 'you', 'know', 'that', 'he', 'was', 'such', 'a', 'sensation', 'that', 'Steve', 'Jobs', 'wanted', 'to', 'build', 'Apple', 'to', 'be', 'like', 'Sony', 'Here', 'are', 'the', 'similarities', 'Both', 'Apple', 'and', 'Sony', 'had', 'a', 'reputation', 'of', 'creating', 'perfectly', 'crafted', 'consumer', 'products', 'which', 'conquer', 'the', 'marketplace', 'Apple', 'still', 'does', 'in', 'fact', 'Both', 'companies', 'were', 'leaders', 'in', 'personal', 'technology', 'This', 'is', 'where', 'Apple', 'drew', 'inspiration', 'from', 'Sony', 's', 'Walkman', 'an', 'inspiration', 'which', 'culminated', 'in', 'the'], ['In', 'the', 'tradition', 'of', 'Phil', 'Knight', 's', 'Shoe', 'Dog', 'comes', 'the', 'incredible', 'untold', 'story', 'of', 'how', 'Netflix', 'went', 'from', 'concept', 'to', 'company', 'all', 'revealed', 'by', 'co', 'founder', 'and', 

In [47]:
word2vec_model = Word2Vec(min_count=2, workers=-1, size=300)
word2vec_model.build_vocab(corpus) # need to modify

In [48]:
word2vec_model.intersect_word2vec_format('GoogleNews-vectors-negative300.bin.gz', lockf=1.0, binary=True)

In [49]:
word2vec_model.train(corpus, total_examples=word2vec_model.corpus_count, epochs=15)

(0, 0)

In [ ]:
word2vec_model.wv.vocab

In [69]:
doc_embedding_list = list()
# for line in df_data['cleaned'][2:3]:
for line in df_data['cleaned']:
  # print(line)
  doc2vec = None
  for word in line.split():
    # print(word)
    if word in word2vec_model.wv.vocab:
      # print(word2vec_model[word])
      if doc2vec is None:
        doc2vec = word2vec_model[word] 
      else :
        doc2vec = doc2vec + word2vec_model[word] 

  if doc2vec is not None:
    doc2vec = doc2vec / len(doc2vec)
    doc_embedding_list.append(doc2vec)
  # print(type(doc2vec), doc2vec.shape, doc2vec)

print(doc_embedding_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
type(doc_embedding_list), doc_embedding_list[3]

## cosin metrix

In [72]:
from sklearn.metrics.pairwise import cosine_similarity

In [73]:
cosine_sim = cosine_similarity(doc_embedding_list, doc_embedding_list)
type(cosine_sim), cosine_sim.shape

(numpy.ndarray, (2381, 2381))

In [74]:
cosine_sim[3]

array([0.7652013 , 0.7288478 , 0.7382637 , ..., 0.7205809 , 0.671141  ,
       0.75326025], dtype=float32)

##predict

In [75]:
title = 'The Da Vinci Code'
title = 'The Four Pillars of Investing' # 3

In [79]:
idx = 3
sim_scores = list(enumerate(cosine_sim[3]))
print(sim_scores)

[(0, 0.7652013), (1, 0.7288478), (2, 0.7382637), (3, 1.0000001), (4, 0.70852065), (5, 0.78052557), (6, 0.8593161), (7, 0.7965761), (8, 0.73945004), (9, 0.72541755), (10, 0.7626427), (11, 0.7512978), (12, 0.72598165), (13, 0.76437527), (14, 0.68264425), (15, 0.7593478), (16, 0.77798283), (17, 0.7627544), (18, 0.747933), (19, 0.72264415), (20, 0.739023), (21, 0.73652476), (22, 0.7103417), (23, 0.7436234), (24, 0.7929562), (25, 0.64623004), (26, 0.74710804), (27, 0.7641606), (28, 0.7267465), (29, 0.7583428), (30, 0.7627836), (31, 0.7457088), (32, 0.70272654), (33, 0.73092747), (34, 0.7818265), (35, 0.7560634), (36, 0.76583326), (37, 0.75778145), (38, 0.76889163), (39, 0.7515075), (40, 0.7727596), (41, 0.7501653), (42, 0.7340446), (43, 0.7826318), (44, 0.7182108), (45, 0.7281456), (46, 0.7454174), (47, 0.78781325), (48, 0.76131535), (49, 0.6651001), (50, 0.7032439), (51, 0.69864947), (52, 0.75292796), (53, 0.72966254), (54, 0.7792649), (55, 0.735143), (56, 0.77239203), (57, 0.7017862), (58

In [83]:
sim_scores_list = sorted(sim_scores, key=lambda x:x[1], reverse=True)

In [85]:
sim_scores_list[1:6]

[(6, 0.8593161),
 (225, 0.8524448),
 (537, 0.84788775),
 (606, 0.84676266),
 (331, 0.8456889)]

In [87]:
df_data.head(7)

,Desc,Unnamed: 0.1,author,genre,image_link,rating,title,cleaned
0,We know that power is shifting: From West to E...,0.0,Moisés Naím,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.63,The End of Power: From Boardrooms to Battlefie...,We know that power is shifting From West to Ea...
1,Following the success of The Accidental Billio...,1.0,Blake J. Harris,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.94,"Console Wars: Sega, Nintendo, and the Battle t...",Following the success of The Accidental Billio...
2,How to tap the power of social software and ne...,2.0,Chris Brogan,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.78,Trust Agents: Using the Web to Build Influence...,How to tap the power of social software and ne...
3,William J. Bernstein is an American financial ...,3.0,William J. Bernstein,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.20,The Four Pillars of Investing,William J Bernstein is an American financial t...
4,Amazing book. And I joined Steve Jobs and many...,4.0,Akio Morita,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.05,Made in Japan: Akio Morita and Sony,Amazing book And I joined Steve Jobs and many ...
5,In the tradition of Phil Knight's Shoe Dog com...,5.0,Marc Randolph,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.28,That Will Never Work: The Birth of Netflix and...,In the tradition of Phil Knight s Shoe Dog com...
6,"Gregory Zuckerman, the bestselling author of T...",6.0,Gregory Zuckerman,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.12,The Man Who Solved the Market: How Jim Simons ...,Gregory Zuckerman the bestselling author of Th...
